In [ ]:
from airnet.dataset import ServiceNetworkDataset
import numpy as np

data = ServiceNetworkDataset()
demand = data.get_demands() / 180

# Sum by rows (total demand per origin)
row_sums = np.floor(demand.sum(axis=1)).astype(int)

# Sum by columns (total demand per destination)
col_sums = np.floor(demand.sum(axis=0)).astype(int)

total_sum_by_location = row_sums + col_sums
# Display results

print("Total Demand Sums per location:\n", total_sum_by_location)

In [ ]:
from matplotlib import pyplot as plt
import pandas as pd

# 2) Create a single-row DataFrame:
#    - Row index name: "Total Demand"
#    - Column names: "Location_0", "Location_1", ...
df_demand_sum = pd.DataFrame(
    [total_sum_by_location],
    index=["Total Demand"],
    columns=[f"V{i}" for i in range(len(total_sum_by_location))],
)

print("Initial DataFrame shape:", df_demand_sum.shape)
print(df_demand_sum)

# 3) Filter out columns with value == 0:
#    We only keep columns whose value is > 0 in this single row.
df_demand_sum_filtered = df_demand_sum.loc[:, (df_demand_sum > 0).all(axis=0)]

print("\nFiltered DataFrame (nonzero demands) shape:", df_demand_sum_filtered.shape)
print(df_demand_sum_filtered)

selected_locations = df_demand_sum_filtered.columns.tolist()
median_demand = df_demand_sum_filtered.stack().median()
mean_demand = df_demand_sum_filtered.stack().mean()

print("Selected location indices:", selected_locations)
print("Median of nonzero demands:", median_demand)
print("Mean of nonzero demands:", mean_demand)

# 4) Draw a histogram of the remaining demands:
#    Since the data is in a single row, we transpose to get them in one column.
df_demand_sum_filtered.T.plot(kind="hist", bins=50, legend=False)
plt.axvline(
    mean_demand,
    color="green",
    linestyle="dashed",
    linewidth=2,
    label=f"Mean: {mean_demand:.2f}",
)
plt.axvline(
    median_demand,
    color="red",
    linestyle="solid",
    linewidth=2,
    label=f"Median: {median_demand:.2f}",
)
plt.legend(loc="upper right")
plt.xlabel("Demand")
plt.title("Demand distribution per day")
plt.show()

In [ ]:
distances = data.get_distances()

N = distances.shape[0]

# Create a list of labels: ['V0', 'V1', ..., 'V(N-1)']
labels = [f"V{i}" for i in range(N)]

# Build a DataFrame with these row and column names
demand_df = pd.DataFrame(demand, index=labels, columns=labels)
demand_matrix_filtered = demand_df.loc[selected_locations, selected_locations]

In [ ]:
# Create a list of labels: ['V0', 'V1', ..., 'V(N-1)']
labels = [f"V{i}" for i in range(N)]

# Build a DataFrame with these row and column names
distances_df = pd.DataFrame(distances, index=labels, columns=labels)

print(distances_df)

distance_matrix_filtered = distances_df.loc[selected_locations, selected_locations]
print("Filtered distance matrix shape:", distance_matrix_filtered.shape)

# 1) Convert the DataFrame to a NumPy array and flatten
distances_array = distances_df.to_numpy().flatten()

# 2) Calculate mean and median
dist_mean = distances_array.mean()
dist_median = np.median(distances_array)

print(f"Mean distance: {dist_mean:.2f} km")
print(f"Median distance: {dist_median:.2f} km")

# Plot a histogram
plt.figure(figsize=(7, 5))
plt.hist(distances_array, bins=20, edgecolor="black")
plt.axvline(
    dist_mean,
    color="yellow",
    linestyle="dashed",
    linewidth=2,
    label=f"Mean: {dist_mean:.2f}",
)
plt.axvline(
    dist_median,
    color="red",
    linestyle="solid",
    linewidth=2,
    label=f"Median: {dist_median:.2f}",
)
plt.legend(loc="upper right")
plt.xlabel("Distance")
plt.ylabel("Frequency")
plt.title("Travel distance distribution")
plt.show()